In [15]:
import os
import psycopg2
import requests
import json
from flask import Flask, jsonify, request
from langchain_groq import ChatGroq
from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
from langchain.chat_models import init_chat_model

import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")


llm = ChatGroq(
  model="moonshotai/kimi-k2-instruct-0905",
  groq_api_key=GROQ_API_KEY,
  temperature=0,
)


In [16]:
def get_db_connection():
    conn = psycopg2.connect(
        host='localhost',
        dbname=os.getenv('POSTGRES_DB'),
        user=os.getenv('POSTGRES_USER'),
        password=os.getenv('POSTGRES_PASSWORD')
    )
    return conn

In [17]:
conn = get_db_connection()
cur = conn.cursor()
cur.execute("SELECT * FROM users;")
users = cur.fetchall()
cur.close()
conn.close()
users
# json.dumps(users)

[]

In [80]:
def get_data(table: str) -> str:
    """Return info of all users from given table"""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute("SELECT * FROM %s;" % (table,))
    content = cur.fetchall()
    cur.close()
    conn.close()
    if content is None:
        return 'No data found'
    return content

def add_user(UserName: str, UserPassword: str, UserType: str, Classification: str) -> str:
    """Function to add a user to the users table"""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute("INSERT INTO users (UserName, UserPassword, UserType, Classification) VALUES (%s, %s, %s, %s);", (UserName, UserPassword, UserType, Classification))
    conn.commit()
    cur.close()
    conn.close()
    return 'User added successfully'

def add_booking(BidderName: str, WorkerName: str, StartDate: str, EndDate: str, HoursBooked: list, HourlyRate: str) -> str:
    """Function to add a booking to the bookings table"""
    for h in range(len(HoursBooked)):
        if len(HoursBooked[h]) == 0:
            HoursBooked[h] = [-1, -1]
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute("INSERT INTO bookings (BidderName, WorkerName, StartDate, EndDate, HoursBooked, HourlyRate) VALUES (%s, %s, %s, %s, %s, %s);", (BidderName, WorkerName, StartDate, EndDate, HoursBooked, HourlyRate))
    conn.commit()
    cur.close()
    conn.close()
    return 'Booking added successfully'

def add_availability(WorkerName: str, AvailableHours: list) -> str:
    """Function to add availability to the availability table when a worker type user adds their available hours"""
    for h in range(len(AvailableHours)):
        if len(AvailableHours[h]) == 0:
            AvailableHours[h] = [-1, -1]
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute("INSERT INTO availability (WorkerName, AvailableHours) VALUES (%s, %s);", (WorkerName, AvailableHours))
    conn.commit()
    cur.close()
    conn.close()
    return 'Availability added successfully'

agent = create_react_agent(
    model=llm,
    tools=[get_data, add_user, add_booking, add_availability],
    prompt="You are a postgres database management interface. If you can do multiple operations, do them all one by one, and return concise outputs."
)


In [ ]:
# add_availability("Alice", [[], [9, 17], [9, 17], [9, 17], [9, 17], [], []])

'Availability added successfully'

In [82]:
agent.invoke(
    {"messages": [{"role": "user", "content": "Get all data from the tables users"}]}
)


{'messages': [HumanMessage(content='Get all data from the tables users', additional_kwargs={}, response_metadata={}, id='43fbe690-2b81-427b-8086-2b01b1cb9e97'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'functions.get_data:0', 'function': {'arguments': '{"table":"users"}', 'name': 'get_data'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 447, 'total_tokens': 464, 'completion_time': 0.062306528, 'prompt_time': 0.023318161, 'queue_time': 0.252639677, 'total_time': 0.085624689, 'prompt_tokens_details': {'cached_tokens': 256}}, 'model_name': 'moonshotai/kimi-k2-instruct-0905', 'system_fingerprint': 'fp_05df423bab', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--754399a6-7acb-4142-bf3f-94e9270ca0f2-0', tool_calls=[{'name': 'get_data', 'args': {'table': 'users'}, 'id': 'functions.get_data:0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 447, 'output_tokens': 17, 't

In [74]:
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": '''{
        "operation": "add user",
        "name": "nexturn",
        "password": "nexturn123",
        "type": "bidder",
        "classification": "tech"
    }'''}]
    }
)

{'messages': [HumanMessage(content='{\n        "operation": "add user",\n        "name": "nexturn",\n        "password": "nexturn123",\n        "type": "bidder",\n        "classification": "tech"\n    }', additional_kwargs={}, response_metadata={}, id='b7f0a3ff-cd05-447f-8ed7-f4f0b5ef5b1e'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'functions.add_user:0', 'function': {'arguments': '{"Classification":"tech","UserName":"nexturn","UserPassword":"nexturn123","UserType":"bidder"}', 'name': 'add_user'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 483, 'total_tokens': 525, 'completion_time': 0.106934898, 'prompt_time': 0.023123579, 'queue_time': 0.25890788, 'total_time': 0.130058477, 'prompt_tokens_details': {'cached_tokens': 256}}, 'model_name': 'moonshotai/kimi-k2-instruct-0905', 'system_fingerprint': 'fp_3312304636', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--54aec9e

In [75]:
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": '''{
        "operation": "add user",
        "name": "xyz",
        "password": "xyz123",
        "type": "worker",
        "AvailableHours": [[], [9,17], [9,17], [9,17], [9,17], [], []],
        "classification": "developer"
    }'''}]
    }
)

{'messages': [HumanMessage(content='{\n        "operation": "add user",\n        "name": "xyz",\n        "password": "xyz123",\n        "type": "worker",\n        "AvailableHours": [[], [9,17], [9,17], [9,17], [9,17], [], []],\n        "classification": "developer"\n    }', additional_kwargs={}, response_metadata={}, id='84e60b61-dc14-4472-8da8-321b8b5c966f'),
  AIMessage(content="I'll add the user and their availability. Let me process this step by step.", additional_kwargs={'tool_calls': [{'id': 'functions.add_user:0', 'function': {'arguments': '{"Classification":"developer","UserName":"xyz","UserPassword":"xyz123","UserType":"worker"}', 'name': 'add_user'}, 'type': 'function'}, {'id': 'functions.add_availability:1', 'function': {'arguments': '{"AvailableHours":[[],[9,17],[9,17],[9,17],[9,17],[],[]],"WorkerName":"xyz"}', 'name': 'add_availability'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 510, 'total_tokens': 614, 'completi

In [55]:
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": '''{
        "operation": "add booking",
        "BidderName": "nexturn",
        "Workername": "xyz",
        "StartDate": 20251010,
        "EndDate": 20251020,
        "HoursBooked": [[], [9,17], [9,17], [9,17], [9,17], [], []],
        "HourlyRate": 100
    }'''}]
    }
)

{'messages': [HumanMessage(content='{\n        "operation": "add booking",\n        "BidderName": "nexturn",\n        "Workername": "xyz",\n        "StartDate": 20251010,\n        "EndDate": 20251020,\n        "HoursBooked": [[], [9,17], [9,17], [9,17], [9,17], [], []],\n        "HourlyRate": 100\n    }', additional_kwargs={}, response_metadata={}, id='a8465c8f-9421-49dd-b98a-ea9cbef4bb15'),
  AIMessage(content="I'll add this booking to the database for you.", additional_kwargs={'tool_calls': [{'id': 'functions.add_booking:0', 'function': {'arguments': '{"BidderName":"nexturn","EndDate":"20251020","HourlyRate":"100","HoursBooked":[[],[9,17],[9,17],[9,17],[9,17],[],[]],"StartDate":"20251010","WorkerName":"xyz"}', 'name': 'add_booking'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 529, 'total_tokens': 627, 'completion_time': 0.240689492, 'prompt_time': 0.037920173, 'queue_time': 0.262683136, 'total_time': 0.278609665, 'prompt_tokens

In [57]:
agent.invoke(
    {"messages": [{"role": "user", "content": "Add a user with name companyname, bidder, cocaine, password is password"}]
    }
)

{'messages': [HumanMessage(content='Add a user with name companyname, bidder, cocaine, password is password', additional_kwargs={}, response_metadata={}, id='dbd8c503-eef5-473d-834e-2646e5787969'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'functions.add_user:0', 'function': {'arguments': '{"Classification":"cocaine","UserName":"companyname","UserPassword":"password","UserType":"bidder"}', 'name': 'add_user'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 455, 'total_tokens': 497, 'completion_time': 0.102549112, 'prompt_time': 0.022381293, 'queue_time': 0.258524835, 'total_time': 0.124930405, 'prompt_tokens_details': {'cached_tokens': 256}}, 'model_name': 'moonshotai/kimi-k2-instruct-0905', 'system_fingerprint': 'fp_3312304636', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7098b55a-e0a1-4382-adfe-8b972c69b1f0-0', tool_calls=[{'name': 'add_user', 'args': {'Classificati

In [56]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": '''{
        "operation": "get",
        "table": "users"
    }'''}]
    }
)
print(response["messages"][-1].content)

Here are all users in the `users` table:

| UserID | UserName | Password   | UserType | Classification |
|--------|----------|------------|----------|------------------|
| 1      | nexturn  | nexturn123 | bidder   | tech             |
| 2      | xyz      | xyz123     | worker   | developer        |


In [147]:
from datetime import datetime, timedelta

def previous_monday(date_int):
    date = datetime.strptime(str(date_int), "%Y%m%d")
    days_to_subtract = (date.weekday())
    previous_monday_date = date - timedelta(days=days_to_subtract)
    return int(previous_monday_date.strftime("%Y%m%d"))

def expand_slots(slots):
    result = []
    for start, end in slots:
        result.extend(range(start, end + 1))
    return result

def expand_slots_duration(slots, basedate, start_date, duration):
    basedate_date = datetime.strptime(str(basedate), "%Y%m%d")
    start_date_date = datetime.strptime(str(start_date), "%Y%m%d")
    end_date_date = datetime.strptime(str(basedate), "%Y%m%d") + timedelta(weeks=duration)
    diff_days = (start_date_date - basedate_date).days*24
    days = end_date_date - start_date_date
    result = []
    for i in range(((end_date_date - basedate_date).days + 1)*24):
        if i >= diff_days:
            for start, end in slots:
                if start <= i % 168 <= end:
                    result.append(i)
    return result


def has_overlap(slots1, slots2):
    return not set(slots1).isdisjoint(set(slots2))

def check_booking_overlap(search_slots, search_start_date, search_duration, booking_slots, booking_start_date, booking_duration):
    basedate = previous_monday(min(search_start_date, booking_start_date))
    search_slots_expanded = expand_slots_duration(search_slots, basedate, search_start_date, search_duration)
    booking_slots_expanded = expand_slots_duration(booking_slots, basedate, booking_start_date, booking_duration)
    return has_overlap(search_slots_expanded, booking_slots_expanded)

def check_admin_overlap(search_slots, admin_slots):
    return has_overlap(expand_slots(search_slots), expand_slots(admin_slots))

In [161]:
def search_booking(Position: str, Slots: list, StartDate: str, Duration: int) -> str:
    """Function to search for bookings based on position and slots"""
    conn = get_db_connection()
    cur = conn.cursor()
    cur.execute("SELECT WorkerName from workers where Position = %s;", (Position,))
    worker = cur.fetchall()
    worker_names = [w[0] for w in worker]
    cur.execute("Select WorkerName, slots from bookings where WorkerName = ANY(%s) and BidderName = 'admin';", (worker_names,))
    slots = cur.fetchall()
    for s in slots:
        if check_admin_overlap(Slots, s[1]):
            worker_names.remove(s[0])
    cur.execute("Select WorkerName, slots, startdate, duration from bookings where WorkerName = ANY(%s) and BidderName != 'admin';", (worker_names,))
    slots = cur.fetchall()
    for s in slots:
        search_slots = Slots
        search_start_date = int(StartDate)
        search_duration = Duration
        booking_slots = s[1]
        booking_start_date = int(s[2].strftime('%Y%m%d'))
        booking_duration = s[3]
        if check_booking_overlap(search_slots, search_start_date, search_duration, booking_slots, booking_start_date, booking_duration):
            worker_names.remove(s[0])
    # print("Available workers:", worker_names)
    cur.close()
    conn.close()
    return json.dumps(worker_names)

search_booking("developer", [[10, 18]], "20250101", 10)

'[]'